# Objectives {.unnumbered}

By the end of this lab, you will:
1. Load and analyze the **Lightcast dataset** in **Spark DataFrame**.
2. Create **five easy and three medium-complexity visualizations** using **Plotly**.
3. Explore **salary distributions, employment trends, and job postings**.
4. Analyze **skills in relation to NAICS/SOC/ONET codes and salaries**.
5. Customize **colors, fonts, and styles** in all visualizations (**default themes result in a 2.5-point deduction**).
6. Follow **best practices for reporting on data communication**.

# Step 1: Load the Dataset {.unnumbered}


In [1]:
#import gdown
#url = "https://drive.google.com/uc?id=1V2GCHGt2dkFGqVBeoUFckU4IhUgk4ocQ"
#gdown.download(url, "lightcast_data.csv", quiet=False)


In [ ]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "vscode"
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
spark = SparkSession.builder.appName("lightcast_data.csv").getOrCreate()

df = spark.read.option("header", "true").option("inferSchema", "true").option("multiLine","true").option("escape", "\"").csv("lightcast_data.csv")

#df.printSchema()
#df.show(5)


# Salary Distribution by Employment Type
- Identify salary trends across different employment types.
- **Filter the dataset**
  - Remove records where **salary is missing or zero**.
- **Aggregate Data**
  - Group by **employment type** and compute salary distribution.
- **Visualize results**
  - Create a **box plot** where:
    - **X-axis** = `EMPLOYMENT_TYPE_NAME`
    - **Y-axis** = `SALARY_FROM`
  - Customize **colors, fonts, and styles** to avoid a **2.5-point deduction**.
- **Explanation:** Write two sentences about what the graph reveals.



In [ ]:
# Your Code for 1st question here
pdf = df.select("EMPLOYMENT_TYPE_NAME", "SALARY").toPandas()
fig = px.box(pdf, x="EMPLOYMENT_TYPE_NAME", y="SALARY", title="Salary Distribution by Employment Type", color_discrete_sequence=["#636EFA"])
fig.update_layout(font_family="Arial", title_font_size=16)
fig.show()

##### This chart shows how salaries vary based on job type. People working full-time (> 32 hours a week) usually earn more and have a bigger range of salaries, including some very high ones. 

#### Part-time workers, whether only part-time or mixed with full-time, tend to earn less and have more similar pay across the board.

# Salary Distribution by Industry
- Compare salary variations across industries.
- **Filter the dataset**
  - Keep records where **salary is greater than zero**.
- **Aggregate Data**
  - Group by **NAICS industry codes**.
- **Visualize results**
  - Create a **box plot** where:
    - **X-axis** = `NAICS2_NAME`
    - **Y-axis** = `SALARY_FROM`
  - Customize colors, fonts, and styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [ ]:
# Your code for 2nd question here

from pyspark.sql.functions import avg, min, max, count
filtered_df = df.filter((col("SALARY_FROM").isNotNull()) & (col("SALARY_FROM") != 0))

industry_df = filtered_df.groupBy("NAICS2_NAME").agg(
    count("*").alias("job_count"))

industry_df.show() 


In [ ]:

import glob
filtered_df = df.filter(
    (col("SALARY_FROM").isNotNull()) & 
    (col("SALARY_FROM") > 0) & 
    (col("NAICS2_NAME").isin("Health Care and Social Assistance", 
                             "Finance and Insurance", 
                             "Information"))
)

salary_df = filtered_df.select("NAICS2_NAME", "SALARY_FROM")
salary_df.write.mode("overwrite").option("header", True).csv("industry_salaries")

files = glob.glob("industry_salaries/part-*.csv")
pdf = pd.concat((pd.read_csv(f) for f in files), ignore_index=True)

pio.renderers.default = "plotly_mimetype"
fig = px.box(
    pdf,
    x="NAICS2_NAME",
    y="SALARY_FROM",
    title="Salary Distribution by Industry",
    color="NAICS2_NAME",
    color_discrete_sequence=["#1f77b4", "#2ca02c", "#d62728"]  
)

fig.update_layout(
    font_family="Calibri",
    font_size=14,
    title_font_size=20,
    xaxis_title="Industry Type",
    yaxis_title="Salary (USD)",
    plot_bgcolor="#fafafa",
    paper_bgcolor="#f5f5f5",
    showlegend=False
)

fig.show()


#### This chart compares salary ranges across three major industries. The Information industry tends to offer the highest salaries, with a wider spread and more high-paying roles. Finance and Insurance shows slightly lower median salaries, but still competitive. Meanwhile, Health Care and Social Assistance generally offers the lowest pay among the three, though it still has some high-salary outliers.

# Job Posting Trends Over Time
- Analyze how job postings fluctuate over time.
- **Aggregate Data**
  - Count job postings per **posted date (`POSTED`)**.
- **Visualize results**
  - Create a **line chart** where:
    - **X-axis** = `POSTED`
    - **Y-axis** = `Number of Job Postings`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [ ]:
#Filtering and grouping job postings by date
import glob
job_post_df = (
    df.filter(col("POSTED").isNotNull())
      .groupBy("POSTED")
      .agg(count("*").alias("job_count"))
)

# grouping the data to CSV and read with pandas
job_post_df.write.mode("overwrite").option("header", True).csv("job_posts")
files = glob.glob("job_posts/part-*.csv")
pdf = pd.concat((pd.read_csv(f) for f in files), ignore_index=True)

#Format and sorting by date
pdf["POSTED"] = pd.to_datetime(pdf["POSTED"])
pdf = pdf.sort_values("POSTED")

pio.renderers.default = "plotly_mimetype"

fig = px.line(
    pdf,
    x="POSTED",
    y="job_count",
    title="Job Postings Trend Over Time",
    markers=True,  # adds visible dots at data points
    color_discrete_sequence=["#FF7F0E"]  
)

fig.update_layout(
    font=dict(family="Segoe UI", size=14),
    title_font=dict(family="Georgia", size=22, color="#333333"),
    xaxis_title="Dates on which the jobs were posted",
    yaxis_title="Number of Job Postings",
    plot_bgcolor="#FAFAFA",
    paper_bgcolor="#FFFFFF"
)

fig.show()


#### This line graph shows how the number of job postings changed from May to October 2024. While the overall trend stays fairly consistent, there are regular ups and downs—suggesting that job postings spike on certain days or weeks, then dip right after. It could reflect typical hiring cycles or patterns like jobs being posted more often early in the week.

# Top 10 Job Titles by Count
- Identify the most frequently posted job titles.
- **Aggregate Data**
  - Count the occurrences of each **job title (`TITLE_NAME`)**.
  - Select the **top 10 most frequent titles**.
- **Visualize results**
  - Create a **bar chart** where:
    - **X-axis** = `TITLE_NAME`
    - **Y-axis** = `Job Count`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [ ]:
# Your code for 4th question here
#finding the top 10 jobs
job_title = df.filter(col("TITLE_NAME").isNotNull()) \
                .groupBy("TITLE_NAME") \
                .agg(count("*").alias("job_count"))
top_10_jobs = job_title.orderBy("job_count", ascending=False).limit(10)

top_10_jobs.show()

In [ ]:

top_10_jobs.write.mode("overwrite").option("header", True).csv("top_10_jobs")
files = glob.glob("top_10_jobs/part-*.csv")
pdf4 = pd.concat((pd.read_csv(f) for f in files), ignore_index=True)

pio.renderers.default = "plotly_mimetype"


fig = px.bar(
    pdf4,
    x="TITLE_NAME",
    y="job_count",
    title="Top 10 Job Titles by Demand",
    color="TITLE_NAME",
    color_discrete_sequence=px.colors.qualitative.Safe  
)

fig.update_layout(
    font=dict(family="Verdana", size=14),
    title_font=dict(family="Georgia", size=22, color="#333333"),
    height=550,
    width=900,
    xaxis_title="Job Titles",
    yaxis_title="Number of Postings",
    plot_bgcolor="#F8F8F8",
    paper_bgcolor="#FFFFFF",
    showlegend=True,
    legend_title="Legend"
)

fig.show()


#### This bar chart shows the top 10 most in-demand job titles based on the number of postings. Data Analysts dominate the chart by a wide margin, indicating very high demand. The rest—including roles like Business Intelligence Analysts, Enterprise Architects, and Oracle Cloud HCM Consultants—have significantly fewer postings, showing that while specialized roles are needed, generalist data positions are currently leading the job market.

# Remote vs On-Site Job Postings
- Compare the proportion of remote and on-site job postings.
- **Aggregate Data**
  - Count job postings by **remote type (`REMOTE_TYPE_NAME`)**.
- **Visualize results**
  - Create a **pie chart** where:
    - **Labels** = `REMOTE_TYPE_NAME`
    - **Values** = `Job Count`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [ ]:
# Your code for 5th question here
remote_jobs = df.filter(col("REMOTE_TYPE_NAME").isNotNull()) \
                .groupBy("REMOTE_TYPE_NAME") \
                .agg(count("*").alias("job_count"))

remote_jobs.show()

In [ ]:

remote_jobs.write.mode("overwrite").option("header", True).csv("remote_jobs")
files = glob.glob("remote_jobs/part-*.csv")
pdf5 = pd.concat((pd.read_csv(f) for f in files), ignore_index=True)

pio.renderers.default = "plotly_mimetype"
custom_colors = ["#00A699", "#FF6F61", "#1E90FF", "#FFD166", "#6D6875", "#F4A261"]

fig = px.pie(
    pdf5,
    names="REMOTE_TYPE_NAME",
    values="job_count",
    title="Remote vs On-Site Job Distribution",
    color_discrete_sequence=custom_colors,
    hole=0.4  
)

fig.update_layout(
    font=dict(family="Segoe UI", size=15),
    title_font=dict(family="Georgia", size=22, color="#222222"),
    showlegend=True,
    legend_title="Work Type",
    paper_bgcolor="#FFFFFF"
)

fig.show()


This donut chart shows how job postings are split between remote, hybrid, and on-site work types. A huge chunk (78.1%) of the postings don’t specify the work type, which could mean missing or incomplete data. Among the rest, remote jobs make up 17.2%, while hybrid and on-site (not remote) roles are much less common at just over 3% and 1.5% respectively. So, while remote work is present, transparency in job type still has room for improvement.

# Skill Demand Analysis by Industry (Stacked Bar Chart)
- Identify which skills are most in demand in various industries.
- **Aggregate Data**
  - Extract **skills** from job postings.
  - Count occurrences of skills grouped by **NAICS industry codes**.
- **Visualize results**
  - Create a **stacked bar chart** where:
    - **X-axis** = `Industry`
    - **Y-axis** = `Skill Count`
    - **Color** = `Skill`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [24]:
from pyspark.sql.functions import col, split, explode, trim, count
import glob
import pandas as pd
import plotly.express as px
import plotly.io as pio

# Step 1: Prepare and write cleaned industry-skill data
skills_df = df.filter(
    col("SKILLS_NAME").isNotNull() &
    col("NAICS2_NAME").isNotNull() &
    col("NAICS2_NAME").isin("Health Care and Social Assistance", 
                            "Finance and Insurance", 
                            "Information")
).withColumn("SKILL", explode(split(col("SKILLS_NAME"), ","))) \
 .withColumn("SKILL", trim(col("SKILL")))

industry_skills_df = skills_df.groupBy("NAICS2_NAME", "SKILL") \
                              .agg(count("*").alias("skill_count"))

industry_skills_df.write.mode("overwrite").option("header", True).csv("industry_skills_cleaned")

# Step 2: Read the CSVs into pandas
files = glob.glob("industry_skills_cleaned/part-*.csv")
pdf_skills = pd.concat((pd.read_csv(f) for f in files), ignore_index=True)

# Step 3: Clean up bad values
pdf_skills["SKILL"] = pdf_skills["SKILL"].astype(str).str.strip()
pdf_skills = pdf_skills[pdf_skills["SKILL"].str.len() > 2]  # remove short/malformed junk

valid_industries = ["Health Care and Social Assistance", "Finance and Insurance", "Information"]
pdf_skills = pdf_skills[pdf_skills["NAICS2_NAME"].isin(valid_industries)]

# Step 4: Get top 10 skills per industry
top_skills = (
    pdf_skills.groupby("NAICS2_NAME")
    .apply(lambda x: x.nlargest(10, "skill_count"))
    .reset_index(drop=True)
)

# Step 5: Plot
pio.renderers.default = "plotly_mimetype"

fig = px.bar(
    top_skills,
    x="SKILL",
    y="skill_count",
    color="NAICS2_NAME",
    title="Top 10 Skills by Industry (Combined)",
    barmode="group",
    color_discrete_sequence=px.colors.qualitative.Set2
)

fig.update_layout(
    font_family="Calibri",
    font_size=13,
    title_font_size=20,
    height=600,
    plot_bgcolor="#ffffff_


SyntaxError: unterminated string literal (detected at line 58) (950956871.py, line 58)


# Salary Analysis by ONET Occupation Type (Bubble Chart)
- Analyze how salaries differ across ONET occupation types.
- **Aggregate Data**
  - Compute **median salary** for each occupation in the **ONET taxonomy**.
- **Visualize results**
  - Create a **bubble chart** where:
    - **X-axis** = `ONET_NAME`
    - **Y-axis** = `Median Salary`
    - **Size** = Number of job postings
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [ ]:
print(f"Number of ONET groups: {len(onet_groups)}")
print(onet_groups[:10])  # Peek at first 10

In [ ]:
subset = onet_df.filter(col("ONET_NAME") == onet)


In [ ]:
# Your code for 7th question here

results = []

for onet in onet_groups:
    subset = onet_df.filter(col("ONET_NAME") == onet)
    job_count = subset.count()

    if job_count >= 5:  # Skip groups with very few jobs
        median_salary = subset.approxQuantile("SALARY_FROM", [0.5], 0.01)[0]
        results.append((onet, median_salary, job_count))
# Create DataFrame and filter out empty/zero values just in case
onet_summary_df = pd.DataFrame(results, columns=["ONET_NAME", "median_salary", "job_count"])
onet_summary_df = onet_summary_df[
    (onet_summary_df["median_salary"] > 0) &
    (onet_summary_df["job_count"] > 0)
]



In [ ]:
import plotly.express as px
import plotly.io as pio

pio.renderers.default = "plotly_mimetype"

fig = px.scatter(
    onet_summary_df,
    x="ONET_NAME",
    y="median_salary",
    size="job_count",
    title="Median Salary vs Job Count by ONET Job Family",
    color="median_salary",
    size_max=60,
    color_continuous_scale="Cividis"
)

fig.update_layout(
    font=dict(family="Segoe UI", size=14),
    title_font=dict(family="Georgia", size=22, color="#2F4F4F"),
    height=600,
    width=1000,
    xaxis_title="ONET Job Family",
    yaxis_title="Median Salary (USD)",
    xaxis_tickangle=45,
    plot_bgcolor="#FAFAFA",
    paper_bgcolor="#FFFFFF",
    coloraxis_colorbar=dict(title="Median Salary")
)

fig.show()



In [ ]:
print(onet_summary_df.head())
print(onet_summary_df.shape)


# Career Pathway Trends (Sankey Diagram)
- Visualize job transitions between different occupation levels.
- **Aggregate Data**
  - Identify career transitions between **SOC job classifications**.
- **Visualize results**
  - Create a **Sankey diagram** where:
    - **Source** = `SOC_2021_2_NAME`
    - **Target** = `SOC_2021_3_NAME`
    - **Value** = Number of transitions
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [ ]:
# Your code for 8th question here